### Import packages

In [1]:
import pystaq
import pyqpp

### Parse some QASM code and expand broadcasted operations

In [18]:
src = "OPENQASM 2.0; \
       include \"qelib1.inc\"; \
       \
       qreg q[3]; \
       creg c[3]; \
       \
       h q[0]; \
       cx q[0],q[1]; \
       cx q[0],q[2]; \
       \
       measure q -> c;"

circuit = pystaq.parse_str(src)
pystaq.desugar(circuit)
circuit

OPENQASM 2.0;
include "qelib1.inc";

qreg q[3];
creg c[3];
h q[0];
cx q[0],q[1];
cx q[0],q[2];
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];

### Create a test device and write it to a json file

In [19]:
device = pystaq.Device(3)
device.add_edge(0, 1)
device.add_edge(1, 2)
device

with open("device.json", "w") as file:
    file.write(str(device))

### Map the circuit onto the device

In [20]:
pystaq.map(circuit, "linear", "swap", device_json="device.json")
circuit

OPENQASM 2.0;
include "qelib1.inc";

qreg q[3];
creg c[3];
U(pi/2,0,pi) q[0];
CX q[0],q[1];
CX q[0],q[1];
CX q[1],q[0];
CX q[0],q[1];
CX q[1],q[2];
measure q[1] -> c[0];
measure q[0] -> c[1];
measure q[2] -> c[2];

### Try again, with a better mapping algorithm

In [21]:
pystaq.map(circuit, "linear", "steiner", device_json="device.json")
circuit

OPENQASM 2.0;
include "qelib1.inc";

qreg q[3];
creg c[3];
U(pi/2,0,pi) q[0];
CX q[1],q[0];
CX q[0],q[1];
CX q[1],q[2];
measure q[1] -> c[0];
measure q[0] -> c[1];
measure q[2] -> c[2];

### Write the circuit to and run 100 simulations

In [24]:
with open("circuit.qasm", "w") as file:
    file.write(str(circuit))
    
qcircuit = pyqpp.qasm.read_from_file("circuit.qasm")
pyqpp.QEngine(qcircuit).execute(100)

[QEngine]
<QCircuit nq: 3, nc: 3, d: 2>
last probs: [0.5, 1, 1]
last dits: [1, 1, 1]
stats:
	reps: 100
	outcomes: 2
	[0 0 0]: 42
	[1 1 1]: 58